In [ ]:
import math
import sys

sys.path.append("../")

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np

from constants import SPEED_OF_SOUND

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

np.random.seed(1)

def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from file_parser import read_recordings, parameters

#dir_name = "recordings_9_7_20"  # audio shield measurements with buffer length 256, only 200Hz source
#dir_name = "recordings_14_7_20"  # audio shield measurements with buffer length 8060, only 200Hz source
dir_name = "recordings_16_7_20"  # recordings with measurement mics, 200Hz and white_noise source

source = "200Hz"
#source = "white_noise"

loudness = "high"
#loudness = "normal"
#loudness = "low"

gt_degrees = 0 # rotation of drone with respect to source.
#gt_degrees = 20 # rotation of drone with respect to source.
#gt_degrees = 40 # rotation of drone with respect to source.

signals_props, signals_source, signals_all = read_recordings(dir_name,
                                                             loudness,
                                                             gt_degrees,
                                                             source)
params = parameters[dir_name]
Fs = params['Fs']
time_index = params['time_index']

mic_idx = [0]
mic_idx = range(4)
#n_buffer = 2**10
n_buffer = signals_props.shape[1] // 2
#n_buffer = 256
assert n_buffer <= signals_props.shape[1]
print(f'length of signal: {n_buffer / Fs:.2f}s')
buffer_props = signals_props[mic_idx, time_index:time_index+n_buffer]
buffer_source = signals_source[mic_idx, time_index:time_index+n_buffer]
buffer_all = signals_all[mic_idx, time_index:time_index+n_buffer]

In [ ]:
from plotting_beamform import plot_signals, get_max_bins

freqs = np.fft.rfftfreq(n_buffer, d=1/Fs)

max_bins = get_max_bins(buffer_props)
plot_signals(buffer_props, Fs=Fs, plot_frequencies=freqs[max_bins])
plt.title('propeller signal')
max_bins = get_max_bins(buffer_source)
plot_signals(buffer_source, Fs=Fs, plot_frequencies=freqs[max_bins]) 
plt.title('source signal')
max_bins = get_max_bins(buffer_all)
plot_signals(buffer_all, Fs=Fs, plot_frequencies=freqs[max_bins])
plt.title('combined signal')

In [ ]:
ipd.Audio(data=buffer_props, rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=buffer_all, rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=buffer_source, rate=Fs, normalize=True)

# Noise cancellation schemes

In [ ]:
def plot_comparison_frequency(original, filtered, title='filtered', phase=False):
    from plotting_beamform import normalize
    fig, axs = plt.subplots(2, sharex=True, sharey=True)
    for m in range(original.shape[0]):
        axs[0].semilogx(freqs, 20*np.log10(np.abs(original[m])), label=f'mic{m}', color=f"C{m}")
        axs[1].semilogx(freqs, 20*np.log10(np.abs(filtered[m])), label=f'mic{m}', color=f"C{m}")
    axs[0].get_xaxis().set_visible(False)
    [ax.set_ylabel('amplitude [dB]') for ax in axs]
    axs[1].set_xlabel('frequency [Hz]')
    axs[0].set_title('original'); axs[1].set_title(title)
    plt.legend()

    if phase:
        fig, axs = plt.subplots(2, sharex=True, sharey=True)
        for m in range(original.shape[0]):
            axs[0].semilogx(freqs, np.angle(original[m]), label=f'mic{m}', color=f"C{m}")
            axs[1].semilogx(freqs, np.angle(filtered[m]), label=f'mic{m}', color=f"C{m}")
        axs[0].get_xaxis().set_visible(False)
        [ax.set_ylabel('phase [rad]') for ax in axs]
        axs[1].set_xlabel('frequency [Hz]')
        axs[0].set_title('original'); axs[1].set_title(title)
        plt.legend()
        
    return fig, axs
    
def plot_comparison_time(original, filtered, title='filtered'):
    from plotting_beamform import normalize
    length = 100
    fig, axs = plt.subplots(2, sharex=True, sharey=True)
    for m in range(original.shape[0]):
        axs[0].plot(normalize(original[m, :length]).T, color=f"C{m}")
        axs[1].plot(normalize(filtered[m, :length]).T, color=f"C{m}")
    axs[0].set_title('original'); axs[1].set_title(title)
    [ax.set_ylabel('amplitude [-]') for ax in axs]
    axs[1].set_xlabel('time index [-]')
    return fig, axs

In [ ]:
filtered_dict = {}
freqs = np.fft.rfftfreq(n_buffer, d=1/Fs)
buffer_all_f = np.fft.rfft(buffer_all)

## IIR bandpass

In [ ]:
from noise_cancellation import filter_iir_bandpass

bin_ = get_max_bins(buffer_source)[0]
f_center = freqs[bin_]
print(bin_, freqs[bin_])

f_delta = 20
order = 10
fmin = f_center - f_delta
fmax = f_center + f_delta
filtered_dict['cheby'] = filter_iir_bandpass(buffer_all, fmin, fmax, Fs, method='cheby2', order=order, plot=True)
filtered_dict['single_peak'] = filter_iir_bandpass(buffer_all, fmin, fmax, Fs, method='single_peak', plot=True)

for name, buffer_filtered in filtered_dict.items():
    buffer_filtered_f = np.fft.rfft(buffer_filtered)
    fig, axs = plot_comparison_frequency(buffer_all_f, buffer_filtered_f, title=name)
    [ax.axvline(x=f, color='k', ls=':') for ax in axs for f in [fmin, fmax]]
    plot_comparison_time(buffer_all, buffer_filtered, title=name)

## Difference

In [ ]:
buffer_props_f = np.fft.rfft(buffer_props)

#abs_props = np.mean(np.abs(buffer_props_f), axis=0)
#abs_props_f = np.abs(buffer_props_f[0])
abs_props_f = np.abs(buffer_props_f[1])
fig, ax = plt.subplots()
ax.set_title('average propellers signal')
ax.semilogx(freqs, 20*np.log10(abs_props_f), label='propellers', color='k')

# replace the absolute part of the signal with the difference 
abs_buffer_diff_f = (np.abs(buffer_all_f) - abs_props_f)
buffer_diff_f = abs_buffer_diff_f * np.exp(1j*np.angle(buffer_all_f))

# test that this gives back the original signal
#buffer_diff_f = np.abs(buffer_all_f) * np.exp(1j*np.angle(buffer_all_f))
buffer_diff = np.fft.irfft(buffer_diff_f, n=buffer_all.shape[1])

filtered_dict['diff'] = buffer_diff
plot_comparison_frequency(buffer_all_f[:1], buffer_diff_f[:1], title='diff')
plot_comparison_time(buffer_all, buffer_diff, title='diff')

## Wiener filter

In [ ]:
buffer_source_f = np.fft.rfft(buffer_source)
psd_source = np.abs(buffer_source_f)**2
psd_noise = np.abs(buffer_props_f)**2

g = psd_source / (psd_source + psd_noise)

filtered_wiener_f = np.multiply(g, buffer_all_f)
filtered_wiener = np.fft.irfft(filtered_wiener_f, n=buffer_all.shape[1])
filtered_dict['wiener'] = filtered_wiener 
plot_comparison_frequency(buffer_all_f, filtered_wiener_f, 'wiener')
plot_comparison_time(buffer_all, filtered_wiener, 'wiener')

## Comb filter

In [ ]:
from noise_cancellation import filter_iir_comb

bins_source = get_max_bins(buffer_source)
bins_prop = get_max_bins(buffer_props)
print([freqs[bin_] for bin_ in bins_prop])

f_prop = freqs[bins_prop[0]]
f_source = freqs[bins_source[0]]
print(f_source)
filters = {
    'notch': f_prop / 2,
    'peak': f_source + f_source/2,
}
Q = 5

for ftype, freq in filters.items():
    plt.figure()
    filtered_dict[ftype] = filter_iir_comb(buffer_all, freq, Fs, ftype=ftype, Q=Q, plot=True)
    plt.legend()
    filtered_f = np.fft.rfft(filtered_dict[ftype])
    plot_comparison_frequency(buffer_all_f, filtered_f, ftype)

In [ ]:
ipd.Audio(data=buffer_all, rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=buffer_source, rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['cheby'], rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['single_peak'], rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['diff'], rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['peak'], rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['notch'], rate=Fs, normalize=True)

In [ ]:
ipd.Audio(data=filtered_dict['wiener'], rate=Fs, normalize=True)

In [ ]:
from algos_basics import get_autocorrelation

R_all = get_autocorrelation(buffer_all)
plt.matshow(np.angle(R_all[bin_]))
plt.title('original')
plt.colorbar()
for name, filtered_buffer in filtered_dict.items():
    R_filtered = get_autocorrelation(filtered_buffer)
    plt.matshow(np.angle(R_filtered[bin_]))
    np.testing.assert_allclose(np.angle(R_filtered[bin_]), np.angle(R_all[bin_]), atol=1e-3)
    print(f"angle error for {name}: {np.linalg.norm(np.angle(R_filtered[bin_]) - np.angle(R_all[bin_]))}")
    plt.colorbar()
    plt.title(name)